### Reproducting figure 7 in Acceleration of the Solar System from Gaia astrometry

In [ ]:
from src.data import data_download # Run Python file to download data

In [1]:
import jax 
import jax.numpy as jnp
import numpy as np
from src.models.vsh_model import*
import pandas as pd
from iminuit import Minuit # to perform least square
from src.models.configuration import*
from src.data.data_utils import*
from tqdm import tqdm
import gc

/root/Document/Gaia_Project/mem97/gaia_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load data
df = load_qso_dataframe()
angles, obs, error = config_data(df)

Separating data in to upper and lower plane ($\delta > 0$ and $\delta<0$ respectivelly)

## Upper Plane:

In [3]:
alpha = angles[0]
delta = angles[1]

mask = delta>=0
ra_upper = alpha[mask]
dec_upper = delta[mask]

angles_upper = np.stack([ra_upper, dec_upper])
obs_upper = obs[:, mask]
error_upper = error[:, mask]

In [4]:
assert len(angles_upper[0]) == len(obs_upper[0])
assert len(obs_upper[0]) == len(error_upper[0])

In [5]:
def eval_loss(theta, lmax):
    return least_square(angles_upper, obs_upper, error_upper, theta, lmax=lmax, grid=False)
eval_loss = jit(eval_loss, static_argnames=['lmax'])

def run_iminuit(lmax, t_bound, s_bound):

    def least_square_wrapper(*theta_flat):
        theta = jnp.array(theta_flat)
        return float(eval_loss(theta, lmax)) 

    total_params = count_vsh_coeffs(lmax) 
    limits = vsh_minuit_limits(lmax, t_bound=t_bound, s_bound=s_bound)
    
    theta_init = jnp.zeros(total_params)    

    m = Minuit(least_square_wrapper, *theta_init)

    m.errordef = Minuit.LEAST_SQUARES
    for i, name in enumerate(m.parameters):
        m.limits[name] = limits[name]

    m.migrad()

    return m

In [ ]:
m_10m = run_iminuit(10, 0.08, 0.04)

In [6]:
m_1m = run_iminuit(1, 0.01, 0.0025) # for l=1

param_m_1m = jnp.array([m_1m.values[name] for name in m_1m.parameters])
std_m_1m = jnp.sqrt(jnp.diag(m_1m.covariance))

del m_1m
gc.collect()
jax.clear_caches()

fitted_thetas = [param_m_1m]
std_theta_fit = [std_m_1m]

for i in tqdm(range(2, 4), desc="Running Minuit fits"): # for l= 2, 3

    m = run_iminuit(i, 0.05, 0.01)

    theta_fit = np.array([m.values[name] for name in m.parameters])
    fitted_thetas.append(theta_fit)
    std_theta_fit.append(np.sqrt(np.diag(m.covariance)))

    del m
    gc.collect()
    jax.clear_caches()

Running Minuit fits:  50%|█████     | 1/2 [02:43<02:43, 163.27s/it]


KeyboardInterrupt: 

In [7]:
for i in tqdm(range(4, 11), desc="Running Minuit fits"): # for l= 4, 5, ..., 10

    m = run_iminuit(i, 0.08, 0.04)

    theta_fit = np.array([m.values[name] for name in m.parameters])
    fitted_thetas.append(theta_fit)
    std_theta_fit.append(np.sqrt(np.diag(m.covariance)))

    del m
    gc.collect()
    jax.clear_caches()

Running Minuit fits:  43%|████▎     | 3/7 [56:32<1:15:23, 1130.75s/it]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

lmax_values = [1,2,3,4,5,6,7,8,9,10]

s10_ls, er10_ls = [fitted_thetas[z][1] for z in range(0,10)], [std_theta_fit[z][1] for z in range(0,10)]
s11r_ls, er11r_ls = [fitted_thetas[x][4] for x in range(0,10)], [std_theta_fit[x][4] for x in range(0,10)]
s11i_ls, er11i_ls = [fitted_thetas[y][5] for y in range(0,10)], [std_theta_fit[y][5] for y in range(0,10)]

C0 = np.sqrt(8*np.pi/3)
C1 = np.sqrt(4*np.pi/3)

gz, gz_error = s10_ls/C0, er10_ls/C0
gx, gx_error = -s11r_ls/C1, er11r_ls/C1
gy, gy_error = s11i_ls/C1, er11i_ls/C1


plt.errorbar(lmax_values, gx, yerr=gx_error, fmt='o', capsize=3, markersize=5, elinewidth=1, label=r'$s_{11}^{R}$-pred')
plt.errorbar(lmax_values, gy, yerr=gy_error, fmt='o', capsize=3, markersize=5, elinewidth=1, label=r'$s_{11}^i$-pred')
plt.errorbar(lmax_values, gz, yerr=gz_error, fmt='o', capsize=3, markersize=5, elinewidth=1, label=r'$s_{10}$-pred')


plt.title("Equatorial Upper Panel")
plt.xlabel(r'$l_{max}$')
plt.ylabel(r'Equatorial Components [$\mu$as/yr]')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xticks(lmax_values)
plt.savefig("plots/main_plots/upper_plane.png", dpi=300, bbox_inches='tight')
plt.show()

## Lower Plane:

In [ ]:
alpha = angles[0]
delta = angles[1]

mask = delta<=0
ra_lower = alpha[mask]
dec_lower = delta[mask]

angles_lower = np.stack([ra_lower, dec_lower])
obs_lower = obs[:, mask]
error_lower = error[:, mask]

In [ ]:
def eval_loss(theta, lmax):
    return least_square(angles_lower, obs_lower, error_lower, theta, lmax=lmax, grid=False)
eval_loss = jit(eval_loss, static_argnames=['lmax'])

def run_iminuit(lmax, t_bound, s_bound):

    def least_square_wrapper(*theta_flat):
        theta = jnp.array(theta_flat)
        return float(eval_loss(theta, lmax)) 

    total_params = count_vsh_coeffs(lmax) 
    limits = vsh_minuit_limits(lmax, t_bound=t_bound, s_bound=s_bound)
    
    theta_init = jnp.zeros(total_params)    

    m = Minuit(least_square_wrapper, *theta_init)

    m.errordef = Minuit.LEAST_SQUARES
    for i, name in enumerate(m.parameters):
        m.limits[name] = limits[name]

    m.migrad()

    return m

In [ ]:
m_1m = run_iminuit(1, 0.01, 0.0025) # for l=1

param_m_1m = jnp.array([m_1m.values[name] for name in m_1m.parameters])
std_m_1m = jnp.sqrt(jnp.diag(m_1m.covariance))

fitted_thetas = [param_m_1m]
std_theta_fit = [std_m_1m]

for i in tqdm(range(2, 4), desc="Running Minuit fits"): # for l= 2, 3

    m = run_iminuit(i, 0.05, 0.01)

    theta_fit = jnp.array([m.values[name] for name in m.parameters])
    fitted_thetas.append(theta_fit)
    std_theta_fit.append(jnp.sqrt(jnp.diag(m.covariance)))

    del m

In [ ]:
for i in tqdm(range(4, 11), desc="Running Minuit fits"): # for l= 4, 5, ..., 10

    m = run_iminuit(i, 0.08, 0.04)

    theta_fit = jnp.array([m.values[name] for name in m.parameters])
    fitted_thetas.append(theta_fit)
    std_theta_fit.append(jnp.sqrt(jnp.diag(m.covariance)))

    del m

In [ ]:
import matplotlib.pyplot as plt

lmax_values = [1,2,3,4,5,6,7,8,9,10]

s10_ls, er10_ls = [fitted_thetas[z][1] for z in range(0,10)], [std_theta_fit[z][1] for z in range(0,10)]
s11r_ls, er11r_ls = [fitted_thetas[x][4] for x in range(0,10)], [std_theta_fit[x][4] for x in range(0,10)]
s11i_ls, er11i_ls = [fitted_thetas[y][5] for y in range(0,10)], [std_theta_fit[y][5] for y in range(0,10)]

C0 = np.sqrt(8*np.pi/3)
C1 = np.sqrt(4*np.pi/3)

gz, gz_error = s10_ls/C0, er10_ls/C0
gx, gx_error = -s11r_ls/C1, er11r_ls/C1
gy, gy_error = s11i_ls/C1, er11i_ls/C1


plt.errorbar(lmax_values, gx, yerr=gx_error, fmt='o', capsize=3, markersize=5, elinewidth=1, label=r'$s_{11}^{R}$-pred')
plt.errorbar(lmax_values, gy, yerr=gy_error, fmt='o', capsize=3, markersize=5, elinewidth=1, label=r'$s_{11}^i$-pred')
plt.errorbar(lmax_values, gz, yerr=gz_error, fmt='o', capsize=3, markersize=5, elinewidth=1, label=r'$s_{10}$-pred')


plt.title("Equatorial Lower Panel")
plt.xlabel(r'$l_{max}$')
plt.ylabel(r'Equatorial Components [$\mu$as/yr]')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.xticks(lmax_values)
plt.savefig("plots/main_plots/lower_plane.png", dpi=300, bbox_inches='tight')
plt.show()